In [23]:
import requests
import pandas as pd
import json
import psycopg2 as ps

In [24]:
keys = json.load(open('keys.json'))
API_KEY = keys['API_KEY']


In [25]:
connection_data = json.load(open('connection_data.json'))
host_name = connection_data['host_name']
dbname = connection_data['dbname']
port = connection_data['port']
username = connection_data['username']
password = connection_data['password']
conn = None

In [26]:
try:
    conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
except ps.OperationalError as err:
    raise err
else:
    print('Connected!')

Connected!


In [9]:
conn.close

<function connection.close>

In [27]:
pageSize = '10'
url = 'https://rebrickable.com/api/v3/lego/sets/?key=' + API_KEY + '&page_size=' + pageSize
response = requests.get(url)
results = response.json()['results']
df = pd.json_normalize(results)
df.head()

,set_num,name,year,theme_id,num_parts,set_img_url,set_url,last_modified_dt
0,001-1,Gears,1965,1,43,https://cdn.rebrickable.com/media/sets/001-1/1...,https://rebrickable.com/sets/001-1/gears/,2018-05-05T20:39:47.277922Z
1,0011-2,Town Mini-Figures,1979,67,12,https://cdn.rebrickable.com/media/sets/0011-2/...,https://rebrickable.com/sets/0011-2/town-mini-...,2021-07-04T19:03:52.273186Z
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,0,https://cdn.rebrickable.com/media/sets/0011-3/...,https://rebrickable.com/sets/0011-3/castle-2-f...,2012-04-01T04:47:31.488559Z
3,0012-1,Space Mini-Figures,1979,143,12,https://cdn.rebrickable.com/media/sets/0012-1/...,https://rebrickable.com/sets/0012-1/space-mini...,2013-12-12T23:12:14.245364Z
4,0013-1,Space Mini-Figures,1979,143,12,https://cdn.rebrickable.com/media/sets/0013-1/...,https://rebrickable.com/sets/0013-1/space-mini...,2013-11-08T20:55:48.506908Z


In [28]:
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS sets (
                    set_num VARCHAR(255) PRIMARY KEY,
                    name TEXT,
                    year INTEGER,
                    theme_id INTEGER,
                    num_parts INTEGER,
                    set_img_url TEXT,
                    set_url TEXT,
                    last_modified_dt DATE
    )""")

    curr.execute(create_table_command)

In [29]:
curr = conn.cursor()

In [30]:
create_table(curr)

In [31]:
conn.commit()